### preparation

In [1]:
# !pip install dashscope
import json
import os
import random
import dashscope
from dashscope.api_entities.dashscope_response import Role


In [ ]:
# obtain DASHSCOPE_API_KEY from environment variable
api_key = os.environ.get('DASHSCOPE_API_KEY')
dashscope.api_key = api_key

# Cases


## 1 emotion analysis

In [ ]:

# encapsulate model response function
def get_response(messages):
    response = dashscope.Generation.call(
        model = 'deepseek-v3',
        messages=messages,
        result_format='message' # return full message object
    )
    return response

# test the function
review = 'This product has excellent sound quality that exceeds expectations.'

messages=[
    {"role": "system", "content": "You are a sentiment analyst. Help me determine whether the product review is positive or negative. Reply with a single word: Positive or Negative."},
    {"role": "user", "content": review}
]

response = get_response(messages)
response.output.choices[0].message.content

## 2 Weather Function

In [ ]:
# Getting Weather information
def get_current_weather(location, unit="Celsius"):
    temperature = -1
    if '北京' in location or 'Beijing' in location:
        temperature = 10
    if '上海' in location or 'Shanghai' in location:
        temperature = 15
    if '深圳' in location or 'Shenzhen' in location:
        temperature = 20

    weather_info = {
        "location": location,
        "temperature": temperature,
        "unit": unit,
        "description": "Partly cloudy",
        "forecast": [
            {"day": "Monday", "description": "Sunny", "high": temperature + 2, "low": temperature - 2},
            {"day": "Tuesday", "description": "Rainy", "high": temperature + 1, "low": temperature - 3},
            {"day": "Wednesday", "description": "Cloudy", "high": temperature + 3, "low": temperature - 1},
        ]
    }
    
    return json.dumps(weather_info)


# Encapsulate the weather function
def get_response(messages):
    try:
        response = dashscope.Generation.call(
            # model = 'deepseek-v3',
            model='qwen-max',
            messages=messages,
            functions=functions,
            result_format='message' # return full message object
        )
        return response
    except Exception as e:
        print(f"Error occurred: {e}")
        return None


# Test the weather function with Q&A
def run_conversation():
    query = "What's the weather like in Beijing today?"
    messages = [{"role":"user", "content": query}]
    response = get_response(messages)
    
    # step1 1st response from model
    if not response or not response.output:
        print("Failed to get response from model.")
        return None
    
    print('response=', response)

    message = response.output.choices[0].message
    messages.append(message) # keep Historical conversations 保留对话历史，保持上下文
    print("messages=", message)

    # Step2 User Call Function 
    if hasattr(message, 'function_call') and message.function_call:
        function_call = message.function_call
        # tool_name = function_call.name
        tool_name = function_call['name']

    # step3 Excute Function
        arguments = json.loads(function_call['arguments'])
        print("arguments=", arguments)
        tool_response = get_current_weather(
            location=arguments.get('location'),
            unit=arguments.get('unit', 'Celsius'),
        )

        tool_info = {
            "role": "function",
            "name": tool_name,
            "content": tool_response
        }
        messages.append(tool_info)
        print("messages after tool response=", messages)

        # step4 obatain 2nd response from model
        response = get_response(messages)
        if not response or not response.output:
            print("Failed to get 2nd response from model after function call.")
            return None

        print('response=', response)
        message = response.output.choices[0].message
        return message
    return message

# Description of the location
functions = [
    {
        "name": "get_current_weather", # 要用哪个工具
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {
                    "type": "string",
                    "enum": ["Celsius", "Fahrenheit"],
                    "description": "The unit of temperature",
                },
            },
            "required": ["location"],
        },
    }
]

if __name__ == "__main__":
    final_message = run_conversation()
    if final_message:
        print("Final response from model:", final_message)
    else:
        print("No final response obtained.")

## 3 Table Extraction

In [ ]:
# encapsulate model response function
def get_response(messages):
    response = dashscope.MultiModalConversation.call(
        model = 'qwen-v1-plus',
        messages=messages
    )
    return response

content = [
    {'image': 'https://aiwucai.oss-cn-huhehaote.aliyuncs.com/pdf_table.jpg'}, # Either a local path or an url
    {'text': '这是一个表格图片，帮我提取里面的内容，输出JSON格式'}
]

messages=[{'role':'user','content':content}]

response = get_response(messages)
response


In [ ]:
print(response.output.choices[0].message.content[0]['text'])

## 4 Operation Incident Handling - 运维事件处理

In [3]:
# 第三方接口获取 database状态
def get_current_status():
    # 连接数
    connections=random.randint(10,100)
    # CPU使用率
    cpu_usage=round(random.uniform(1, 100), 1)
    # 内存使用率
    memory_usage=round(random.uniform(1, 100), 1)

    status_info = {
        "connections": connections,
        "cpu_usage": f"{cpu_usage}%",
        "memory_usage": f"{memory_usage}%"
    }
    
    return json.dumps(status_info, ensure_ascii=False)

# Encapsulate the status function
def get_response(messages):
    try:
        response = dashscope.Generation.call(
            model='qwen-turbo',
            messages=messages,
            tools=tools,
            result_format='message' # return full message object
        )
        return response
    except Exception as e:
        print(f"Error occurred: {e}")
        return None
    

current_locals=locals()
current_locals

tools = [
    {
        "type": "function",
            "function": {
                "name": "get_current_status",
                "description": "调用监控系统接口，获取当前数据库服务器性能指标，包括：连接数、CPU使用率、内存使用率",
                "parameters": {
                },
                "required": []
            }       
    }
]

query = """告警：数据库连接数超过设定阈值
时间：2024-08-03 15:30:00
"""

messages = [
    {"role": "system", "content": "我是运维分析师，用户会告诉我们告警内容。我会基于告警内容，判断当前的异常情况（告警对象、异常模式）"},
    {"role":"user", "content": query}
]




In [4]:
while True:
    response = get_response(messages)
    if not response or not getattr(response, 'output', None) or not getattr(response.output, 'choices', None):
        print('Bad response:',
              getattr(response, 'status_code', None),
              getattr(response, 'code', None),
              getattr(response, 'message', None))
        break

    choice = response.output.choices[0]
    message = getattr(choice, 'message', None)
    messages.append(message)

    if getattr(choice, 'finish_reason', None) == 'stop':
        break

    # step2 User Call Function
    # 兼容 tool_calls 和 function_call 两种形态
    if isinstance(message, dict):
        tool_calls = message.get('tool_calls')
        function_call = message.get('function_call')
    else:
        tool_calls = getattr(message, 'tool_calls', None)
        function_call = getattr(message, 'function_call', None)

    if tool_calls:
        call = tool_calls[0]
        fn = call['function']
        fn_name = fn['name']
        fn_arguments = fn.get('arguments') or "{}"
        call_id = call.get('id') or call.get('tool_call_id')
        arguments_json = json.loads(fn_arguments)

        # step3 Execute Function
        function = current_locals[fn_name]
        tool_response = function(**arguments_json)
        tool_info = {
            "role": "tool",
            "tool_call_id": call_id,
            "name": fn_name,
            "content": tool_response
        }
        # print("tool_info=", tool_info)
        messages.append(tool_info)

    elif function_call:
        fn_name = function_call['name']
        fn_arguments = function_call.get('arguments') or "{}"
        arguments_json = json.loads(fn_arguments)

        function = current_locals[fn_name]
        tool_response = function(**arguments_json)
        tool_info = {
            "role": "function",
            "name": fn_name,
            "content": tool_response
        }
        messages.append(tool_info)
    else:
        break

Error occurred: HTTPSConnectionPool(host='dashscope.aliyuncs.com', port=443): Max retries exceeded with url: /api/v1/services/aigc/text-generation/generation (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000029B5943D130>, 'Connection to dashscope.aliyuncs.com timed out. (connect timeout=300)'))
Bad response: None None None


In [5]:
messages

[{'role': 'system',
  'content': '我是运维分析师，用户会告诉我们告警内容。我会基于告警内容，判断当前的异常情况（告警对象、异常模式）'},
 {'role': 'user', 'content': '告警：数据库连接数超过设定阈值\n时间：2024-08-03 15:30:00\n'}]

## 5 Emotion analysis - Deepseek

In [3]:
# ecapulate model response function
def get_response(messages):
    response = dashscope.Generation.call(
        model = 'deepseek-r1',
        messages=messages,
        result_format='message' # return full message object
    )
    return response

# test the function
messages=[
    {"role": "system", "content": "You are a helpful assistant "},
    {"role": "user", "content": "你好，你是什么大模型？"}
]

response = get_response(messages)
print(response.output.choices[0].message.content)

你好呀！我是 **DeepSeek-R1**，是由中国人工智能公司 **深度求索（DeepSeek）** 研发的一款大语言模型助手。你可以叫我“小深”～😊

我具备强大的文本理解、推理和创作能力，可以帮你处理各种任务，比如：
- 解答知识问题 📚
- 写作、翻译、润色文章 ✍️
- 编程辅助、代码生成 💻
- 学习辅导、考试复习 📖
- 聊天陪伴、情绪支持 🌸

而且我完全免费！目前你可以通过网页或者 App 来使用我，非常方便。

有什么我可以帮你的吗？😊


## 6 Searching Website

In [20]:
# %pip install -U "typing_extensions>=4.12.2" "openai>=1.0.0"
# !pip install openai
# !pip show typing_extensions openai
from openai import OpenAI

Name: typing_extensions
Version: 4.15.0
Summary: Backported and Experimental Type Hints for Python 3.9+
Home-page: 
Author: 
Author-email: "Guido van Rossum, Jukka Lehtosalo, Łukasz Langa, Michael Lee" <levkivskyi@gmail.com>
License: 
Location: e:\Python\python3.12\Lib\site-packages
Requires: 
Required-by: aiosignal, anyio, formulaic, openai, pydantic, pydantic_core, typing-inspection
---
Name: openai
Version: 2.14.0
Summary: The official Python library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: Apache-2.0
Location: e:\Python\python3.12\Lib\site-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: 


In [30]:
# api_key = os.environ.get('DASHSCOPE_API_KEY')

client = OpenAI(
    api_key ="sk-16f2c80d9d3e4d7d9cc2f6b43f22dfbd",  # 填写你的DashScope API Key
    # 填写DashScope服务的base_url
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",  
)

completion = client.chat.completions.create(
    model="qwen-plus",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello, who won the world series in 2020?"}
    ],
    extra_body={
        "enable_search": True
    }
)

print(completion.model_dump_json())

{"id":"chatcmpl-94b8b730-a050-43a5-94f9-4eba05e72507","choices":[{"finish_reason":"stop","index":0,"logprobs":null,"message":{"content":"Hello! The Los Angeles Dodgers won the World Series in 2020. They defeated the Tampa Bay Rays in six games (4-2) to claim their seventh championship in franchise history. The series was played at a neutral site—Globe Life Field in Arlington, Texas—due to the pandemic-related adjustments that year.","refusal":null,"role":"assistant","annotations":null,"audio":null,"function_call":null,"tool_calls":null}}],"created":1766394292,"model":"qwen-plus","object":"chat.completion","service_tier":null,"system_fingerprint":null,"usage":{"completion_tokens":67,"prompt_tokens":33,"total_tokens":100,"completion_tokens_details":null,"prompt_tokens_details":{"audio_tokens":null,"cached_tokens":0}}}
